## Algoritmo Genético

No jupyter anterior utilizamos o algoritmo Gradiente Decrescente para otimizar a função objetivo, afim de comparações, dessa vez utilizaremos um algoritmo genético.

In [1]:
from random import randint, uniform, random
from tqdm import tqdm
import numpy as np

In [2]:
# Manipulação das matrizes

def somar(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] - B[i][j]

    return C


#################################################################

## d (psi) / dt = -i * H * |psi>
def dpsidt(t, psi, H): # A derivada da onda em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H) # i * H
    return -1*np.matmul(A,psi) # - i * H * |psi>

# Runge-Kutta de quarta ordem

def rungeKutta(onda, fatorRungeKutta, hamiltoniano, tempoFinal = 2, tempoInicial = 0):
    
    resultado = np.zeros((2,2), dtype=np.complex_)
    indice = 0

    for i in range(0, 2):
        
        resultado[0][indice] = onda[0][0]
        resultado[1][indice] = onda[1][0]
        
        k1 = dpsidt(tempoInicial, onda, hamiltoniano)
        k2 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k1)), hamiltoniano)
        k3 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k2)), hamiltoniano)
        k4 = dpsidt(tempoInicial + fatorRungeKutta, somar(onda, np.dot(fatorRungeKutta, k3)), hamiltoniano)
        
        ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
        
        A = somar(np.dot(2,k3), k4)
        B = somar(np.dot(2,k2), k1)
        C = somar(A, B)
 
        onda = somar(onda,np.dot((fatorRungeKutta / 6.0),(C)))
    
        tempoInicial += fatorRungeKutta
        
        indice += 1
        
    runge = np.zeros((2,1), dtype=np.complex_)
    runge[0][0] = resultado[0][indice-1]
    runge[1][0] = resultado[1][indice-1]
    return runge

def funcaoObjetivo(x, iteracao, ondaDestino, hamiltoniano, onda, horizonteAnalisavel, fatorRungeKutta, tempoAnalise):
    
    avanco = np.zeros((2,1),dtype=np.complex_)
    
    y = 0
    
    horizonte = (horizonteAnalisavel+iteracao)/100
    
    tempo = iteracao/100
    
    tempoInicialAnalise = 0.00
    tempoFinalAnalise = 0.02
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, x[0]], [x[0], 0]]
    
    matrizOrigem = np.zeros((2,1),dtype=np.complex_)
    matrizOrigem = [[onda[0][0]], [onda[1][0]]] 

    matrizDestino = np.zeros((2,1),dtype=np.complex_)
    
    ## Função-Objetivo (Return) = somatorio ||(Matriz_Origem - Matriz_Destino)||^2
    
    ## Avanço temporal
    
    matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
    matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)

    y += (np.linalg.norm(matrizOrigem-matrizDestino))**2

    avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

    matrizOrigem = avanco

    tempoInicialAnalise += 0.01
    tempoFinalAnalise += 0.01
    
    for indice in range((iteracao+1), (horizonteAnalisavel+iteracao)):
        tempo = indice/100
        
        matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
        matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)
        
        y += (np.linalg.norm(matrizOrigem-matrizDestino))**2
        
        controles = [[0, x[int(tempoInicialAnalise*100)]], [x[int(tempoInicialAnalise*100)], 0]]

        avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

        matrizOrigem = avanco

        tempoInicialAnalise += 0.01
        tempoFinalAnalise += 0.01
    
    return y

In [3]:
# Parâmetros iniciais

tamanhoPopulacao = 30
taxaMutacao = 0.01
taxaCrossover = 0.85
numeroGeracoes = 70
quantidadeVariaveis = 3 # Para o sistema dinâmico avaliado, apenas os controles devem ser avaliados
quantidadeTrocas = 2 # Posicoes a serem trocadas na 

In [4]:
def gerarPopulacao(xMin, xMax):
    populacao = np.zeros((tamanhoPopulacao,quantidadeVariaveis))
    for individuo in range(tamanhoPopulacao):
        for pos in range(quantidadeVariaveis):
            populacao[individuo,pos] = uniform(xMin, xMax)
    return populacao

In [5]:
def divisaoFitness(valorFuncaoObjetivo):
    divisao = 0
    for i in range(len(valorFuncaoObjetivo)):
        divisao += valorFuncaoObjetivo[i]
    return divisao

def fitness(valorFuncaoObjetivo):
    return valorFuncaoObjetivo/divisaoFitness(valorFuncaoObjetivo)

In [6]:
def calcularFuncao(populacao, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise):
    valorFuncaoObj = np.zeros(tamanhoPopulacao)
    for individuo in range(tamanhoPopulacao):
        valorFuncaoObj[individuo] = funcaoObjetivo(populacao[individuo,:], iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    return valorFuncaoObj

In [7]:
def selecao(valorFitness):
    pos = np.random.choice(range(tamanhoPopulacao), 2, replace=False)
    valor = valorFitness[pos[0]] < valorFitness[pos[1]]
    if valor == True:
        return pos[0]
    else:
        return pos[1]

In [8]:
def cruzamento(pai1, pai2):
    x = random()
    filho1 = x*pai1+(1-x)*pai2
    filho2 = (1-x)*pai1+x*pai2
    return filho1, filho2

In [9]:
def mutacao(individuo):
    pos = np.random.choice(range(quantidadeVariaveis),  quantidadeTrocas ,replace=False)
    individuo[pos[0]] = individuo[pos[1]]
    individuo[pos[1]] = individuo[pos[0]]
    return individuo

In [10]:
import matplotlib.pyplot as plt

In [11]:
def main(xMin, xMax, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise):
    pop = gerarPopulacao(xMin, xMax)
    fo = calcularFuncao(pop, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    fit = fitness(fo)
    melhor_fo = np.zeros(numeroGeracoes)
    melhor_ind_geracao = np.zeros((numeroGeracoes, quantidadeVariaveis))
    for geracao in range(numeroGeracoes):
        new_pop = np.empty((0, quantidadeVariaveis))
        for i in range(round((tamanhoPopulacao) / 2)):
            pai1 = 0
            pai2 = 0
            while pai1 == pai2:
                pai1 = selecao(fit)
                pai2 = selecao(fit)
            if random() < taxaCrossover:
                filho1, filho2 = cruzamento(pop[pai1, :], pop[pai2, :]) 
                if random() < taxaMutacao:    
                    filho1 = mutacao(filho1)
                    filho2 = mutacao(filho2)  
                new_pop = np.vstack([new_pop, filho1, filho2])     
            else:
                pais = np.vstack([pop[pai1, :], pop[pai2, :]])
                new_pop = np.vstack([new_pop, pais])
        pop = new_pop    
        fo = calcularFuncao(pop, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
        fit = fitness(fo)
        min_fo = min(fo)
        melhor_fo[geracao] = min_fo
        pos_min_fo = np.where(fo == min_fo)

    
    #plt.plot(melhor_fo)
    #plt.xlim(0, numeroGeracoes)
    #plt.ylabel('Gerações')
    #plt.ylabel('Melhor Custo')
    #plt.title('Evolução da função objetivo - GA')
    #plt.grid(True)
    #plt.show()
    

    return pop[pos_min_fo,:][0,0]

In [12]:
# Função de origem e destino
ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

hamiltoniano = np.zeros((2,2), dtype=np.complex_) # Hamiltoniano
A = (1/2*np.pi)
B = (3/2*np.pi)
hamiltoniano = [[A, 0], [0, B]]

iteracao = 0
fatorRungeKutta = 0.01
tempoAnalise = 2000
horizonte = 3

xMin = -10
xMax = 10

constantes4 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1

minimo = main(xMin, xMax, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)

In [13]:
minimo

array([0.12617049, 0.36219368, 0.36219368])

In [14]:
while iteracao < 2000:
    
    print(f"Iteração: {iteracao}")
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, minimo[0]],[minimo[0],0]]
    hamiltoniano = somar(hamiltoniano, controle)

    constantes4[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes4[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)

    print(ondaOrigem[0][0])
    print(ondaOrigem[1][0])
    print("--------------")
    
    ## Adaptação da onda ao controle

    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)

    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]

    xMax = -minimo[0]
    xMin = minimo[0]
    
    iteracao += 1

    minimo = main(xMax, xMin, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)

Iteração: 0
(0.8+0j)
(0.6+0j)
--------------
Iteração: 1
(0.7998768917802795-0.013322455218528814j)
(0.5993017441756494-0.029272675821119074j)
--------------
Iteração: 2
(0.7994584243628865-0.02723416561794642j)
(0.5971692532302483-0.05927403147055064j)
--------------
Iteração: 3
(0.7986976133434125-0.041549350797967094j)
(0.5935566073810389-0.08970132397565021j)
--------------
Iteração: 4
(0.7975562886404146-0.056188865180090634j)
(0.5884315549911346-0.12039552606448248j)
--------------
Iteração: 5
(0.7960318963191135-0.07097125782252658j)
(0.58178811446525-0.15105922719592824j)
--------------
Iteração: 6
(0.7941202853785331-0.08582328418371832j)
(0.5736293358699457-0.18153986019911447j)
--------------
Iteração: 7
(0.791810415512631-0.10072276544943821j)
(0.5639601433004738-0.21175492122616515j)
--------------
Iteração: 8
(0.7891097029697184-0.11560711958589458j)
(0.5527990236055329-0.24156595266840444j)
--------------
Iteração: 9
(0.786022949043578-0.13044040905012202j)
(0.5401685132

Iteração: 75
(0.21127099658040172-0.5841589765937553j)
(-0.6826355836084427+0.38487856510024404j)
--------------
Iteração: 76
(0.20398461390559822-0.5842346303540467j)
(-0.6665279810335775+0.4156929374114671j)
--------------
Iteração: 77
(0.1968410441463801-0.584276406340178j)
(-0.648986172709634+0.4457483882318618j)
--------------
Iteração: 78
(0.18983700625119607-0.5842932075978354j)
(-0.6300473778580916+0.4749775298150488j)
--------------
Iteração: 79
(0.18296876729316003-0.5842936988125216j)
(-0.6097519898637603+0.5033148200274354j)
--------------
Iteração: 80
(0.17623215545473814-0.5842862845391497j)
(-0.5881434845843895+0.530696711898068j)
--------------
Iteração: 81
(0.1696225740447504-0.5842790880933727j)
(-0.5652683216175773+0.5570617986171267j)
--------------
Iteração: 82
(0.1631350165146323-0.5842799311533315j)
(-0.5411758387505502+0.5823509536516388j)
--------------
Iteração: 83
(0.15676408243861098-0.5842963141188674j)
(-0.5159181398339645+0.6065074656581474j)
------------

Iteração: 148
(-0.344959722914661-0.6305464924778323j)
(0.5607072999771168-0.4111219063178262j)
--------------
Iteração: 149
(-0.3568405256141623-0.6276283384385384j)
(0.5377879473102578-0.43535229346359344j)
--------------
Iteração: 150
(-0.3687848365291896-0.6244122174893522j)
(0.5137684280363277-0.45842024049289976j)
--------------
Iteração: 151
(-0.38078231835691145-0.6208921529986661j)
(0.48870614917572547-0.48027497081357856j)
--------------
Iteração: 152
(-0.39282231021367076-0.6170626099348931j)
(0.4626608719347779-0.5008685186517833j)
--------------
Iteração: 153
(-0.40489384822781294-0.6129185105818837j)
(0.4356945745277057-0.5201558382581538j)
--------------
Iteração: 154
(-0.41698568687157594-0.6084552492652516j)
(0.4078713099370263-0.5380949064301137j)
--------------
Iteração: 155
(-0.42908632098411614-0.6036687060552413j)
(0.37925705893769707-0.5546468181160222j)
--------------
Iteração: 156
(-0.44118400843615313-0.598555259414094j)
(0.34991957872112744-0.5697758748827221

Iteração: 220
(-0.737812618788813+0.19767382150141585j)
(-0.2289965367664224+0.6034220477338936j)
--------------
Iteração: 221
(-0.7317270624732103+0.21022719926772285j)
(-0.199272730146041+0.6169849198824023j)
--------------
Iteração: 222
(-0.7253855546656464+0.22254102758974445j)
(-0.16888502291404922+0.629101834906897j)
--------------
Iteração: 223
(-0.7187988141801398+0.23460840000568395j)
(-0.1379034411582466+0.6397419680928532j)
--------------
Iteração: 224
(-0.7119778019428618+0.24642291436050753j)
(-0.10639941548030865+0.6488778795452608j)
--------------
Iteração: 225
(-0.7049336968575116+0.25797868266676827j)
(-0.07444561755983695+0.6564855776221519j)
--------------
Iteração: 226
(-0.6976778712439552+0.26927033980437626j)
(-0.042115793870608216+0.6625445744828323j)
--------------
Iteração: 227
(-0.6902218659060522+0.28029305104049895j)
(-0.009484596929203282+0.667037933623667j)
--------------
Iteração: 228
(-0.6825773648854311+0.2910425183534702j)
(0.023372585538160233+0.66995

Iteração: 293
(-0.20574149564840163+0.5595964655358151j)
(0.13101353951271147-0.7920591270758646j)
--------------
Iteração: 294
(-0.20068129965982218+0.5622520023743052j)
(0.09622705853259615-0.7964546631837593j)
--------------
Iteração: 295
(-0.19559565706927415+0.5649925070458504j)
(0.061284306843097804-0.7992308897164334j)
--------------
Iteração: 296
(-0.1904755721785725+0.5678180657171318j)
(0.02626213610760275-0.8003823937321823j)
--------------
Iteração: 297
(-0.18531203499989793+0.5707282597897972j)
(-0.008762434126621005-0.7999074309987311j)
--------------
Iteração: 298
(-0.18009604652323352+0.5737221654957423j)
(-0.04371239181909307-0.7978079297805664j)
--------------
Iteração: 299
(-0.17481864397862185+0.5767983547175339j)
(-0.07851090780669967-0.7940894862407676j)
--------------
Iteração: 300
(-0.16947092603443228+0.5799548970323928j)
(-0.11308151075341683-0.7887613514678917j)
--------------
Iteração: 301
(-0.16404407787297073+0.583189362975336j)
(-0.14734826127937456-0.781

...

Iteração: 1670
(0.6684283213105257+0.14667725966536804j)
(-0.28875066583331116+0.6695612550340938j)
--------------
Iteração: 1671
(0.6738446862178252+0.13742179993143278j)
(-0.256293492197996+0.6792364051108694j)
--------------
Iteração: 1672
(0.6791557895531298+0.1279279133147957j)
(-0.22346143237845134+0.6873475911637053j)
--------------
Iteração: 1673
(0.6843504647187592+0.11819574811479397j)
(-0.19032968837483535+0.6938794394807688j)
--------------
Iteração: 1674
(0.6894174875536832+0.1082259851551991j)
(-0.15697409970497378+0.6988201887286792j)
--------------
Iteração: 1675
(0.6943456018466225+0.09801984108941036j)
(-0.12347097000322567+0.702161716125677j)
--------------
Iteração: 1676
(0.6991235449954257+0.08757907048232531j)
(-0.08989689256904984+0.7038995552963218j)
--------------
Iteração: 1677
(0.703740073754078+0.07690596666303173j)
(-0.05632857526409435+0.7040329057670611j)
--------------
Iteração: 1678
(0.7081839900084826+0.0660033613452752j)
(-0.02284266515811812+0.702564

Iteração: 1743
(0.35182323655558867-0.7001406433038906j)
(-0.10917824930817216-0.6116399709672482j)
--------------
Iteração: 1744
(0.3379087111215785-0.7049666589438752j)
(-0.1412238346286281-0.6073675950922254j)
--------------
Iteração: 1745
(0.32394516814508423-0.7094228834100573j)
(-0.1730529436860161-0.6015240921503451j)
--------------
Iteração: 1746
(0.3099458044222416-0.7135098510460524j)
(-0.20459005735684788-0.5941212446584496j)
--------------
Iteração: 1747
(0.29592373978138287-0.7172286580256392j)
(-0.23576030070072349-0.5851744415240249j)
--------------
Iteração: 1748
(0.2818919913460547-0.7205809574936016j)
(-0.26648961476061966-0.574702643210052j)
--------------
Iteração: 1749
(0.26786344806768214-0.723568953497431j)
(-0.2967049265028525-0.5627283387253571j)
--------------
Iteração: 1750
(0.25385084558513243-0.7261953937242772j)
(-0.3263343165080657-0.5492774945387098j)
--------------
Iteração: 1751
(0.2398667414676791-0.7284635610602554j)
(-0.35530718402973704-0.534379495

Iteração: 1815
(-0.3192389252894634-0.5029916968037124j)
(0.3138697111782737+0.7393047506316595j)
--------------
Iteração: 1816
(-0.32365433139002725-0.4995005014069555j)
(0.34601727639866603+0.7252716839859812j)
--------------
Iteração: 1817
(-0.3280852952707241-0.496090042684544j)
(0.3774848829180895+0.7097603257104736j)
--------------
Iteração: 1818
(-0.3325399580412778-0.49275663752867704j)
(0.40820327800334905+0.6928044489096652j)
--------------
Iteração: 1819
(-0.3370262322239045-0.48949615366631755j)
(0.4381048960765218+0.6744410889124429j)
--------------
Iteração: 1820
(-0.34155177958511085-0.48630402175532916j)
(0.46712401257934955+0.6547104589937455j)
--------------
Iteração: 1821
(-0.3461239895809869-0.48317524854075505j)
(0.4951968936347231+0.6336558588399962j)
--------------
Iteração: 1822
(-0.35074995846618057-0.48010443104054984j)
(0.5222619411632463+0.6113235759674472j)
--------------
Iteração: 1823
(-0.35543646911521515-0.4770857717277101j)
(0.5482598331232559+0.587762

Iteração: 1887
(-0.7617224671185641-0.04612740675106236j)
(-0.3403576610507246-0.5493702306771878j)
--------------
Iteração: 1888
(-0.7648864798906462-0.03246922520965217j)
(-0.36595939580344794-0.529120010197467j)
--------------
Iteração: 1889
(-0.7677361823014617-0.018645374704831928j)
(-0.3905135958102985-0.5076735940192578j)
--------------
Iteração: 1890
(-0.7702635250847618-0.004665935002304111j)
(-0.41396435242737856-0.4850831763730557j)
--------------
Iteração: 1891
(-0.7724608729704177+0.00945863297117242j)
(-0.4362583271216752-0.46140356308956954j)
--------------
Iteração: 1892
(-0.7743210225583715+0.023717506197080712j)
(-0.4573448733486995-0.4366920455935535j)
--------------
Iteração: 1893
(-0.7758372192779774+0.03809951946906724j)
(-0.47717615226518023-0.4110082692162349j)
--------------
Iteração: 1894
(-0.7770031733938493+0.05259318608707428j)
(-0.49570724201229394-0.3844140961275535j)
--------------
Iteração: 1895
(-0.7778130750214978+0.06718671929546247j)
(-0.51289624031

Iteração: 1959
(-0.21186817266384655+0.6877839344686831j)
(0.6281848261717903+0.29571742830825715j)
--------------
Iteração: 1960
(-0.19973433274679434+0.6880110465051j)
(0.6446875435149771+0.2666923390370639j)
--------------
Iteração: 1961
(-0.1877379713956308+0.6879739711846891j)
(0.6598037722474268+0.23686512484616334j)
--------------
Iteração: 1962
(-0.17588679924782713+0.6876815650719038j)
(0.6734960557739068+0.20630268504243437j)
--------------
Iteração: 1963
(-0.16418806554778975+0.6871429714839015j)
(0.6857301737960084+0.1750736434252044j)
--------------
Iteração: 1964
(-0.1526485460471226+0.6863675976773713j)
(0.6964752212021498+0.14324819168372016j)
--------------
Iteração: 1965
(-0.14127453201553408+0.6853650914993309j)
(0.70570367938275+0.11089792920030121j)
--------------
Iteração: 1966
(-0.13007182038649695+0.6841453175556849j)
(0.7133914798073896+0.07809569962534703j)
--------------
Iteração: 1967
(-0.11904570505916882+0.6827183329524285j)
(0.7195180597184596+0.044915424